# DATA VISUALIZATION WITH BOKEH

In [1]:
import numpy as np
import pandas as pd
from bokeh.io import output_notebook, push_notebook
from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.plotting import ColumnDataSource
from bokeh.layouts import row,column,gridplot,widgetbox
from bokeh.models.widgets import Tabs, Panel,Button,Dropdown,CheckboxGroup,\
                                RadioGroup,Slider,TextInput,Select
from bokeh.models import CategoricalColorMapper
from numpy.random import random, normal
from bokeh.io import curdoc
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
output_notebook()

Loading BokehJS ...

In [2]:
import yaml

from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Slider
from bokeh.plotting import figure
from bokeh.themes import Theme
from bokeh.io import show, output_notebook

from bokeh.sampledata.sea_surface_temperature import sea_surface_temperature

output_notebook()


Loading BokehJS ...

In [62]:
def bkapp(doc):
    df = sea_surface_temperature.copy()
    source = ColumnDataSource(data=df)

    plot = figure(x_axis_type='datetime', y_range=(0, 25),
                  y_axis_label='Temperature (Celsius)',
                  title="Sea Surface Temperature at 43.18, -70.43")
    plot.line('time', 'temperature', source=source)

    def callback(attr, old, new):
        if new == 0:
            data = df
        else:
            data = df.rolling('{0}D'.format(new)).mean()
        source.data = ColumnDataSource.from_df(data)

    slider = Slider(start=0, end=30, value=0, step=1, title="Smoothing by N Days")
    slider.on_change('value', callback)

    doc.add_root(column(slider, plot))
    
    '''
    doc.theme = Theme(json=yaml.load("""
        attrs:
            Figure:
                background_fill_color: "#DDDDDD"
                outline_line_color: white
                toolbar_location: above
                height: 500
                width: 800
            Grid:
                grid_line_dash: [6, 4]
                grid_line_color: white
    """, Loader=yaml.FullLoader))
    '''

In [63]:
show(bkapp) # notebook_url="http://localhost:8888"

In [94]:
def myapp(doc):
    points = 10
    df=pd.DataFrame({'x': random(points), 'y': random(points)})
    source=ColumnDataSource(data=df)
    
    plot = figure()
    plot.circle('x', 'y', source=source)

    def callback(attr, old, new):
        points=slider.value #new
        df=pd.DataFrame({'x': random(points), 'y': random(points)})
        source.data=ColumnDataSource.from_df(df)
        #source.data = ColumnDataSource.from_df(data)
        
        
    slider = Slider(start=10, end=1000, value=10, step=10, title="Number of Points")
    slider.on_change('value', callback)
    #plot.circle('x', 'y', source=source)
    doc.add_root(column(slider, plot))

In [95]:
show(myapp)

In [126]:
def myapp1(doc):
    x = [1,2,3,4,5,6,7,8,9]
    y = [2,3,4,5,6,7,8,9,10]
    df=pd.DataFrame({'x': x, 'y': y})
    source=ColumnDataSource(data=df)
    
    plot = figure()
    plot.line('x', 'y', source=source)

    def callback(attr, old, new):
        points=slider.value #new
        df=pd.DataFrame({'x': random(points), 'y': random(points)})
        source.data=ColumnDataSource.from_df(df)
        #source.data = ColumnDataSource.from_df(data)
        
        
    slider = Slider(start=10, end=1000, value=10, step=10, title="Number of Points")
    slider.on_change('value', callback)
    #plot.circle('x', 'y', source=source)
    doc.add_root(row(slider, plot))

In [127]:
show(myapp1)

# Application with the select widget

In [3]:
initial_points = 500
data_points = ColumnDataSource(data = {'x': random(initial_points), 'y': random(initial_points)})
#Create the plot
plot = figure(title = "Scatter plot distribution selector")
plot.diamond(x = 'x', y = 'y', source = data_points, color = 'red')
#Create the select widget
select_widget = Select(options = ['uniform distribution', 'normal distribution'], value = 'uniform distribution', title = 'Select the distribution of your choice')
#Define the callback function
def callback(attr, old, new):
    if select_widget.value == 'uniform distribution':
        function = random
    else:
        function = normal
    data_points.data = {'x': function(size = initial_points), 'y': function(size = initial_points)}
#select_widget.on_change('value', callback)
#Create a layout for the application
layout = column(select_widget, plot)
#Add the layout to the application
#curdoc().add_root(layout)

def modify_doc(doc):
    doc.add_root(row(layout, width=800))
    doc.title = "Sliders"
    select_widget.on_change('value', callback)


handler = FunctionHandler(modify_doc)
app = Application(handler)
show(app)


# Creating an application with the button widget

In [4]:
#Create data for the plot
initial_points = 500
data_points = ColumnDataSource(data = {'x': random(initial_points), 'y': random(initial_points)})
#Create the plot
plot = figure(title = "Data change application")

plot.diamond(x = 'x', y = 'y', source = data_points, color = 'red')

#Create the button widget
button_widget = Button(label = 'Change Data')
#Define the callback function
def callback():
    #New y values 
    y = np.sin(random(size=initial_points)) + random(size=initial_points)
    data_points.data = {'x': np.cos(random(size=initial_points)), 'y': np.sin(random(size=initial_points))}

button_widget.on_click(callback)
#Create a layout for the application
layout = column(button_widget, plot)    
    
def modify_doc(doc):
    doc.add_root(layout)
    doc.title = "Sliders"
    button_widget.on_click(callback)


handler = FunctionHandler(modify_doc)
app = Application(handler)
show(app)

In [5]:
df = pd.read_csv('all_stocks_5yr.csv')

#Filtering for apple stocks

df_apple = df[df['Name'] == 'AAL']

#Create the ColumnDataSource object

data = ColumnDataSource(data = {
    'x' : df_apple['high'],
    'y' : df_apple['low'],
    'x1': df_apple['volume']
    
})

#Creating the scatter plot 

plot = figure(title = 'Attribute selector application')

plot.diamond('x', 'y', source = data, color = 'red')

#Creating the select widget

select_widget = Select(options = ['low', 'volume'], value = 'low', title = 'Select a new y axis attribute')

#Define the callback function

def callback(attr, old, new):
    
    if new == 'low':
        
        data.data = {'x' : df_apple['high'], 'y': df_apple['low']}
        
    else:
        
        data.data = {'x' : df_apple['high'], 'y': df_apple['volume']}
        
select_widget.on_change('value', callback)


#Add the layout to the application

layout = row(select_widget, plot)

def modify_doc(doc):
    doc.add_root(layout)
    doc.title = "Sliders"
    select_widget.on_change('value', callback)


handler = FunctionHandler(modify_doc)
app = Application(handler)
show(app)

In [6]:
############ START BOILERPLATE ############
#### Interactivity -- BOKEH
import bokeh.plotting.figure as bk_figure
from bokeh.io import curdoc, show
from bokeh.layouts import row, widgetbox
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Slider, TextInput
from bokeh.io import output_notebook # enables plot interface in J notebook
import numpy as np
# init bokeh

from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler


output_notebook()
############ END BOILERPLATE ############

# Set up data
N = 200
x = np.linspace(0, 4*np.pi, N)
y = np.sin(x)
source = ColumnDataSource(data=dict(x=x, y=y))

# Set up plot
plot = bk_figure(plot_height=400, plot_width=400, title="my sine wave",
              tools="crosshair,pan,reset,save,wheel_zoom",
              x_range=[0, 4*np.pi], y_range=[-2.5, 2.5])

plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

# Set up widgets
text = TextInput(title="title", value='my sine wave')
offset = Slider(title="offset", value=0.0, start=-5.0, end=5.0, step=0.1)
amplitude = Slider(title="amplitude", value=1.0, start=-5.0, end=5.0, step=0.1)
phase = Slider(title="phase", value=0.0, start=0.0, end=2*np.pi)
freq = Slider(title="frequency", value=1.0, start=0.1, end=5.1, step=0.1)

# Set up callbacks
def update_title(attrname, old, new):
    plot.title.text = text.value



def update_data(attrname, old, new):
    # Get the current slider values
    a = amplitude.value
    b = offset.value
    w = phase.value
    k = freq.value

    # Generate the new curve
    x = np.linspace(0, 4*np.pi, N)
    y = a*np.sin(k*x + w) + b

    source.data = dict(x=x, y=y)
    ### I thought I might need a show() here, but it doesn't make a difference if I add one
    # show(layout)

for w in [offset, amplitude, phase, freq]:
    w.on_change('value', update_data)


# Set up layouts and add to document
inputs = widgetbox(text, offset, amplitude, phase, freq)
layout = row(plot,
             widgetbox(text, offset, amplitude, phase, freq))



def modify_doc(doc):
    doc.add_root(row(layout, width=800))
    doc.title = "Sliders"
    text.on_change('value', update_title)


handler = FunctionHandler(modify_doc)
app = Application(handler)
show(app)

Loading BokehJS ...